In [18]:
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv
from constants import BTC, ETH, BTCTEST
import bit
from bit.network import NetworkAPI
from web3 import Web3
from web3.middleware import geth_poa_middleware
import os
import lit
from web3 import Web3, middleware, Account
from bit import PrivateKeyTestnet
from web3.gas_strategies.time_based import medium_gas_price_strategy

In [26]:
# Load and set mnemonic variables
load_dotenv()
mnemonic = os.getenv("mnemonic")

In [20]:
# Connects Private Ethereum Proof-of-Authority Blockchain
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.isConnected()

True

In [21]:
w3.middleware_onion.inject(geth_poa_middleware, layer = 0)

In [22]:
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [23]:
# Create a function called 'derive_wallets'
def derive_wallets(mnemonic = mnemonic, coin = BTC, depth = 3):
    command = f'php ./derive -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={depth} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [24]:
# Create a dictionary object called coins to store the output from 'derive_wallets'
coins = {BTCTEST: derive_wallets(coin = BTCTEST), ETH: derive_wallets(coin = ETH)}

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [27]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, private_key):
    if coin == ETH:
        return Account.privateKeyToAccount(private_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(private_key)

In [28]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, to, amount):
    if coin == ETH:
        value = w3.toWei(amount, "ether") # convert 1.2 ETH to 120000000000 wei
        gasEstimate = w3.eth.estimateGas({ "to": to, "from": account, "amount": value })
        return {
            "to": to,
            "from": account,
            "value": value,
            "gas": gasEstimate,
            "gasPrice": w3.eth.generateGasPrice(),
            "nonce": w3.eth.getTransactionCount(account),
            "chainId": w3.eth.chain_id
        }
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [29]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, to, amount):
    if coin == ETH:
        raw_tx = create_tx(coin, account.address, to, amount)
        signed = account.signTransaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    if coin == BTCTEST:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)

In [30]:
derive_wallets()

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [31]:
derive_wallets(coin = ETH)

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [45]:
derive_wallets(coin = BTCTEST)

[{'path': "m/44'/1'/0'/0/0", 'address': 'mhEv1iivKLuMA8m1UqvZ2ECc8KmVcRTBfD', 'xprv': 'tprv8jQpDLKs6Zran4Z89qyQ3csv48EqURxp8qL4S3o2vsE9ciavHsRSA9Dc2SSSmkVJh4ZAqknCc92LXRcN7CtB7G8c84t6LtEGvNjZHsiWgKM', 'xpub': 'tpubDG6rMkN7EwYFfXav3VdzT2Y2d9kmdm9ii8vqiZqLM92YTCqgvGF2LdqUCbi9AcxXwAvtRMLiEhjrptT87sU6PR74QqPxcEwds9rvsXB3iuy', 'privkey': 'cPcRA8kWMGafMs1wFbXxZfzjKFtbTVNxMasHW94hd886pULPoYrD', 'pubkey': '036ea379f84da7c6ee685bc8e9450693c77885be9f832ba7de18707b7d07113778', 'pubkeyhash': '12e7b10b64aa521da314a83b4fccaebc0cc5d953', 'index': 0}, {'path': "m/44'/1'/0'/0/1", 'address': 'mgeqePgiNWmkGaZjbhZamLZa28PvhzxZBA', 'xprv': 'tprv8jQpDLKs6Zraq42e52ESaffPLpiBw4WoM3jykoXJBHEEpwGAsg5wn5XWSMNgNjFjrwMhooKp5pVYy4mour1ENQxEvYxuqorykbrpZffM58p', 'xpub': 'tpubDG6rMkN7EwYFiX4Rxfu2z5KVurE86PhhvMLm3KZbbZ2dfRWwW4uXxa9NcTZcVaGNVbWhM45Kmbtn74pg6uzncyV82jogf2wvoJuCFdN2pge', 'privkey': 'cRizrE6JaCb2ftZrpj9HaHRhRUJKaxjAcCFqicg89esKa7GG6RHF', 'pubkey': '0217cb712efa3edce0907a446082d928a053f2f6dd181e22a1273dfc7